#Implementing Burrows-Wheeler Transformation (BWT) and Decoding for Compression
**Overview of Burrows-Wheeler Transformation**
The Burrows-Wheeler Transformation is a preprocessing step in compression algorithms like bzip2. It reorders the input text to create long runs of repeated characters, which makes it more suitable for
compression using run-length encoding or entropy coding.
****
**Steps of BWT**


1.    **Input String**: Start with the original string and append a special end-of-string character (e.g., $)
that is lexicographically smaller than all other characters
2.    **Create Rotations**: Generate all cyclic rotations of the string.
3.  **Sort Rotations**: Sort these rotations lexicographically.
4.  **Output the Last Column**: Extract the last character of each sorted rotation to form the BWT
output.

****
**Decoding the BWT**
The decoding process reconstructs the original string from the transformed string.

**Steps of Decoding**
1. Input the BWT String: Start with the BWT string.
2. Initialize the Table: Create an empty table with as many rows as the length of the string.
3. Iteratively Sort: Insert the BWT string as a new column to the table and sort the rows
lexicographically. Repeat until the table is complete.
4. Identify Original String: Find the row ending with the special end-of-string character ($). Read
off the characters row-wise to get the original string.




## **BWT**

In [ ]:
# imports
from pprint import pprint
from prettytable import PrettyTable
from tabulate import tabulate
import heapq


# Decorator :
class Decorator:
    def __init__(self, field_name, param1=""):
        self.field_name = field_name
        self.param1 = param1

    def dec1(self, end="\n"):
        print("+", end="")
        for i in range(len(self.field_name) + 6):
            print("-", end="")
        print("*", end="")

        if self.param1 != "":
            for i in range(len(self.param1) + 6):
                print("-", end="")
            print("+", end=end)

    def dec2(self, end=""):
        print("|", self.field_name, end=end, sep="    ")
        if self.param1 != "":
            print("  :", self.param1, "|", sep="   ")

    def full_decorator(self):
        self.dec1()
        self.dec2()
        self.dec1()


class BWT:
    # Constructor :
    def __init__(self, input_str):
        self.input_str = input_str
        self.rotated = False
        self.sorted = False
        self.converted = False
        self.str1 = ["$"]
        self.str2 = [input_str]
        self.bwt_li = [input_str]
        self.sorted_bwt_li = []
        self.matrix = []
        self.first = dict()
        self.last = ()
        self.ranks = []
        self.s_freq = dict()
        self.bwt_output = ""

    # Checks whether the input string contains only one $ or not
    def is_valid(self):
        ds_count = 0
        for i in self.input_str:
            if i == "$":
                ds_count += 1

        if ds_count == 1:
            return True

        return False

    # Create rotations for the given string and store it in self.bwt_li if the string is valid
    def create_rotation(self):
        if self.is_valid():
            new_input_str = self.input_str[0:len(self.input_str) - 1]
            for i in range(len(new_input_str)):
                str1 = self.input_str[:i + 1]
                str2 = self.input_str[i + 1:len(self.input_str) - 1]
                self.str1.append(str1 + "$")
                self.str2.append("$" + str2)
                rotated_str = str2 + "$" + str1
                self.bwt_li.append(rotated_str)
            self.rotated = True

    # Sort the created rotations and store it in self.sorted_bwt_li, if only the string is valid and was rotated
    def sort_rotation(self):
        if self.is_valid() and self.rotated:
            self.sorted_bwt_li = sorted(self.bwt_li)
            self.sorted = True

    # Finding the actual value of BWT (output) from the sorted bwt list
    def find_bwt_output(self):
        if self.rotated and self.sorted:
            for i in self.sorted_bwt_li:
                self.bwt_output += i[len(self.input_str) - 1]

            return self.bwt_output

    # functions to show the results
    def print_table(self, show_bwt_output=False, show_str1=False, show_str2=False):

        if self.rotated:
            table = PrettyTable()
            table.add_column("Rotations", column=self.bwt_li)
            if self.sorted:
                table.add_column("Sorted Rotations", column=self.sorted_bwt_li)

            if show_bwt_output:
                dec = Decorator("BWT Output", self.bwt_output)
                dec.full_decorator()

            if show_str1:
                table.add_column("STR1", column=self.str1)

            if show_str2:
                table.add_column("STR2", column=self.str2)

            print(table)

    def print_matrix(self, show_sorted=True, show_rotated=False, show_str1=False, show_str2=False):
        if self.sorted and show_sorted:
            print("Sorted Matrix")
            print(tabulate(self.sorted_bwt_li, showindex=True), end="\n\n")

        if self.rotated and show_rotated:
            print("Rotations Matrix")
            print(tabulate(self.bwt_li, showindex=True), end="\n\n")

        if show_str1:
            print("Str1 Matrix")
            print(tabulate(self.str1, showindex=True), end="\n\n")
        if show_str2:
            print("Str2 Matrix")
            print(tabulate(self.str2, showindex=True), end="\n\n")

    # Converting the sorted rotations to matrix
    def converting_matrix(self, show=False):
        for i in self.sorted_bwt_li:
            self.matrix.append(list(i))

        if show:
            pprint(self.matrix)
        self.converted = True

        return self.matrix

    # Ranking the characters from the BWT output based on their occurrences
    def rank_bwt(self):
        for char in self.bwt_output:
            if char not in self.s_freq:
                self.s_freq[char] = 0
            self.ranks.append(self.s_freq[char])
            self.s_freq[char] += 1
        return self.ranks, self.s_freq

    def first_col(self):
        c_freq = 0
        for c, count in sorted(self.s_freq.items()):
            self.first[c] = (c_freq, c_freq + count)
            c_freq += count

        return self.first

    def reverse_bwt(self):
        # running the rank_bwt and first_col methods to get the first column and ranking of the characters
        self.rank_bwt()
        self.first_col()

        # Starting at the position of '$' in bwt_output
        row_i = self.bwt_output.index('$')
        reconstructed_str = ""

        # Rebuild the original string by iterating until we have all characters (it equals to length of input string)
        for _ in range(len(self.input_str)):
            # Append the current character (last column) to the reconstructed string
            reconstructed_str = self.bwt_output[row_i] + reconstructed_str

            # Find the next row index by using the rank of the character in the first column
            char = reconstructed_str[0]
            row_i = self.first[char][0] + self.ranks[row_i]

        return reconstructed_str

In [ ]:
bwt = BWT("banana$")
bwt.create_rotation()
bwt.sort_rotation()
bwt.find_bwt_output()
bwt.print_table(show_bwt_output=True, show_str1=True, show_str2=True)
print()
bwt.converting_matrix(show=False)
print(bwt.reverse_bwt())

+----------------*-------------+
|    BWT Output  :   annb$aa   |
+----------------*-------------+
+-----------+------------------+---------+---------+
| Rotations | Sorted Rotations |   STR1  |   STR2  |
+-----------+------------------+---------+---------+
|  banana$  |     $banana      |    $    | banana$ |
|  anana$b  |     a$banan      |    b$   |  $anana |
|  nana$ba  |     ana$ban      |   ba$   |  $nana  |
|  ana$ban  |     anana$b      |   ban$  |   $ana  |
|  na$bana  |     banana$      |  bana$  |   $na   |
|  a$banan  |     na$bana      |  banan$ |    $a   |
|  $banana  |     nana$ba      | banana$ |    $    |
+-----------+------------------+---------+---------+

banana$
{'a': 3, 'n': 2, 'b': 1, '$': 1}


#**Run-Length Encoding (RLE)**
****
Run-Length Encoding is a simple compression algorithm often used in conjunction with BWT to take
advantage of the repeated characters in the transformed string. It works by replacing sequences of the
same character with a single character and a count of its repetitions.
****
**Steps of RLE**
1. Traverse the String: Iterate through the string, keeping track of consecutive occurrences of each
character.
2. Output Compressed Form: For each sequence of repeated characters, output the character
followed by the count of repetitions
****
**Example**
* Input: aaabbbbcc
* RLE Output: a3b4c2

**Advantages**
* Works best for strings with long runs of repeated characters (e.g., BWT output).
* Simple to implement and efficient for specific data types.


## **RLE**

In [ ]:
class RLE:
    def __init__(self, input_str):
        self.input_str = input_str
        self.rle_output_ = ""

    def rle_encode(self):
        encoded_str = []
        count = 1

        for i in range(1, len(self.input_str)):
            if self.input_str[i] == self.input_str[i-1]:
                count+=1
            else:
                encoded_str.append(f"{count}{self.input_str[i-1]}")
                count = 1
        encoded_str.append(f"{count}{self.input_str[-1]}")

        self.rle_output_ = ''.join(encoded_str)

    def rle_decode(self):
        decoded_str = []
        count = ''

        for char in self.rle_output_:
            if char.isdigit():
                count += char
            else:
                decoded_str.append(char * int(count))
                count = ''

        return ''.join(decoded_str)

In [ ]:
bwt.bwt_output

'annb$aa'

In [ ]:
rle = RLE(bwt.bwt_output)
rle.rle_encode()
rle.rle_output_

'1a2n1b1$2a'

In [ ]:
rle.rle_decode()

'annb$aa'

#**Huffman Coding**
Huffman Coding is an entropy-based compression algorithm that assigns variable-length codes to
characters based on their frequencies. More frequent characters get shorter codes.
****
**Steps of Huffman Coding**
1. Build Frequency Table: Count the frequency of each character in the input string.
2. Build Huffman Tree: Construct a binary tree where each leaf node represents a character, and
the path from root to leaf determines its code.
3. Generate Codes: Assign binary codes to characters based on the tree structure.
4. Encode the String: Replace each character in the input with its corresponding code
****
**Advantages**
* Provides optimal compression for a given set of character frequencies.
* Frequently used in conjunction with BWT for high-performance compression algorithms.


## **Huffman**

In [ ]:
! pip install bitarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 19.1 MB/s eta 0:00:00


In [ ]:
from heapq import heappush, heappop, heapify
from collections import defaultdict
from bitarray import bitarray

class Huffman:
    def __init__(self, input_str):
        self.input_str = input_str
        self.huffman_dict = dict()
        self.encoded_text = bitarray()
        self.decoded_text = ""

    def frequency(self):
        freq = defaultdict(int)
        for char in self.input_str:
            freq[char] += 1
        return freq

    def huffman_tree(self):
        freq = self.frequency()
        heap = [[fq, [sym, ""]] for sym, fq in freq.items()]
        heapify(heap)
        print(heap)
        while len(heap) > 1:
          # heappop - Pop and return the smallest item from the heap
          right = heappop(heap)
          left = heappop(heap)

          for pair in right[1:]:
              # add zero to all the right
              pair[1] = '0' + pair[1]
          for pair in left[1:]:
              # add one to all the left
              pair[1] = '1' + pair[1]

          # add values onto the heap. Eg. h = []; heappush(h, (5, 'write code')) --> h = [(5, 'write code')]
          heappush(heap, [right[0] + left[0]] + right[1:] + left[1:])

          huffman_list = right[1:] + left[1:]

        self.huffman_dict = {a[0]:bitarray(str(a[1])) for a in huffman_list}
        print(self.huffman_dict)

    def huffman_encode(self):
        self.encoded_text.encode(self.huffman_dict, self.input_str)

    def huffman_decode(self):
        decoded_text = self.encoded_text.decode(self.huffman_dict)
        self.decoded_text = ''.join(decoded_text)

In [ ]:
huf = Huffman("bbbbbnaass$")

In [ ]:
huf.huffman_tree()

[[1, ['$', '']], [1, ['n', '']], [2, ['a', '']], [2, ['s', '']], [5, ['b', '']]]
{'b': bitarray('0'), 's': bitarray('10'), '$': bitarray('1100'), 'n': bitarray('1101'), 'a': bitarray('111')}


In [ ]:
huf.huffman_encode()
huf.encoded_text

bitarray('00000110111111110101100')

In [ ]:
huf.huffman_decode()
huf.decoded_text

'bbbbbnaass$'